In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import re
import csv
from sqlite3 import dbapi2 as sq3
import os

In [2]:
train = pd.read_csv("train.csv", header = 0)
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S


In [6]:
PATHSTART="."
def get_db(dbfile):
    sqlite_db = sq3.connect(os.path.join(PATHSTART, dbfile))
    return sqlite_db
def init_db(dbfile, schema):
    """Creates the database tables."""
    db = get_db(dbfile)
    db.cursor().executescript(schema)
    db.commit()
    return db
def make_query(sel):
    c=db.cursor().execute(sel)
    return c.fetchall()

In [7]:
ourschema="""
CREATE TABLE "train_table" (
    "PassengerId" INTEGER PRIMARY KEY NOT NULL ,
    "Survived" INTEGER ,
    "Pclass" INTEGER ,
    "Name" VARCHAR ,
    "Sex" VARCHAR ,
    "Age" INTEGER ,
    "SibSp" INTEGER ,
    "ParCh" INTEGER ,
    "Ticket" VARCHAR ,
    "Fare" FLOAT , 
    "Cabin" VARCHAR ,
    "Embarked" VARCHAR
);
"""

In [8]:
db =init_db("train.db", ourschema)
#db.to_sql("train", db, if_exists="append", index=False)

In [9]:
train.to_sql("train", db, if_exists = "append", index = False)

In [39]:
sel = """
SELECT Sex, Survived, COUNT(Survived) from train
GROUP BY Survived, Sex;
"""
make_query(sel)

[(u'female', 0, 81), (u'male', 0, 468), (u'female', 1, 233), (u'male', 1, 109)]

In [46]:
sel = """
SELECT INSTR(Name, ".") as Title from train;
"""
make_query(sel)

[(11,),
 (13,),
 (16,),
 (14,),
 (10,),
 (10,),
 (13,),
 (16,),
 (13,),
 (12,),
 (16,),
 (14,),
 (16,),
 (14,),
 (14,),
 (13,),
 (13,),
 (13,),
 (19,),
 (16,),
 (11,),
 (12,),
 (14,),
 (11,),
 (14,),
 (13,),
 (9,),
 (12,),
 (14,),
 (13,),
 (15,),
 (13,),
 (12,),
 (12,),
 (10,),
 (14,),
 (10,),
 (9,),
 (20,),
 (20,),
 (11,),
 (12,),
 (11,),
 (14,),
 (14,),
 (11,),
 (11,),
 (17,),
 (11,),
 (20,),
 (15,),
 (14,),
 (12,),
 (16,),
 (10,),
 (12,),
 (11,),
 (10,),
 (11,),
 (16,),
 (15,),
 (12,),
 (11,),
 (14,),
 (12,),
 (17,),
 (9,),
 (11,),
 (16,),
 (9,),
 (11,),
 (14,),
 (9,),
 (17,),
 (9,),
 (9,),
 (12,),
 (11,),
 (17,),
 (14,),
 (12,),
 (15,),
 (16,),
 (11,),
 (12,),
 (15,),
 (9,),
 (14,),
 (14,),
 (12,),
 (15,),
 (15,),
 (12,),
 (9,),
 (10,),
 (12,),
 (16,),
 (15,),
 (12,),
 (11,),
 (15,),
 (12,),
 (10,),
 (14,),
 (15,),
 (12,),
 (18,),
 (9,),
 (10,),
 (12,),
 (11,),
 (13,),
 (11,),
 (14,),
 (14,),
 (14,),
 (12,),
 (11,),
 (11,),
 (16,),
 (12,),
 (10,),
 (11,),
 (13,),
 (10,),
 (22,),
 (

In [14]:
train.Sex[train.Survived == 1].groupby(train.Sex).count()

Sex
female    233
male      109
dtype: int64

In [117]:
test = pd.read_csv("test.csv", header = 0)
test_copy = test.copy()
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [3]:
train['FamSize'] = train['SibSp'] + train['Parch'] + 1
train['CabinCategory'] = pd.Series(train['Cabin']).str[0]
cabin_mode = train['CabinCategory'].dropna().mode()[0]
train['CabinCategory'] = train['CabinCategory'].replace(np.nan, cabin_mode)
train['CabinNumber'] = pd.Series(train['Cabin']).dropna().str.extract(r'(\d+)')
train['CabinNumber'] = train['CabinNumber'].replace(np.nan, 0)
avg_age = round(train['Age'].dropna().mean())
train['Age'] = train['Age'].replace(np.nan, avg_age)
train['Title'] = pd.Series(train['Name']).dropna().str.extract(r'([^\W]+\.)')
train['Child'] = np.where(pd.Series(train['Age']) < 18, 1, 0)
sex_mode = train['Sex'].dropna().mode()[0]
train['Sex'] = train['Sex'].replace(np.nan, sex_mode)
train['Sex'] = train['Sex'].replace(['male','female'],[0, 1])

In [4]:
test['FamSize'] = test['SibSp'] + test['Parch'] + 1
test['CabinCategory'] = pd.Series(test['Cabin']).str[0]
cabin_mode = test['CabinCategory'].dropna().mode()[0]
test['CabinCategory'] = test['CabinCategory'].replace(np.nan, cabin_mode)
test['CabinNumber'] = pd.Series(test['Cabin']).dropna().str.extract(r'(\d+)')
test['CabinNumber'] = test['CabinNumber'].replace(np.nan, 0)
avg_age = round(test['Age'].dropna().mean())
test['Age'] = test['Age'].replace(np.nan, avg_age)
test['Title'] = pd.Series(test['Name']).dropna().str.extract(r'([^\W]+\.)')
test['Child'] = np.where(pd.Series(test['Age']) < 18, 1, 0)
sex_mode = test['Sex'].dropna().mode()[0]
test['Sex'] = test['Sex'].replace(np.nan, sex_mode)
test['Sex'] = test['Sex'].replace(['male','female'],[0, 1])

In [92]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamSize,CabinCategory,CabinNumber,Title,Child
0,1,0,3,"Braund, Mr. Owen Harris",0,22,1,0,A/5 21171,7.2500,NaN,S,2,2,0,12,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38,1,0,PC 17599,71.2833,C85,C,2,2,85,13,0
2,3,1,3,"Heikkinen, Miss. Laina",1,26,0,0,STON/O2. 3101282,7.9250,NaN,S,1,2,0,9,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35,1,0,113803,53.1000,C123,S,2,2,123,13,0
4,5,0,3,"Allen, Mr. William Henry",0,35,0,0,373450,8.0500,NaN,S,1,2,0,12,0


In [89]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation
from sklearn import preprocessing
from sklearn.cross_validation import KFold
from sklearn.metrics import accuracy_score
import scipy
import matplotlib as mpl
from sklearn.linear_model import LogisticRegression

In [114]:
from sklearn.grid_search import GridSearchCV
clfl2=RandomForestClassifier()
parameters = {"n_estimators": [10, 100, 200, 300, 400, 500],
              "max_depth": [1, 2, 3, 4, 5, 6]}
fitmodel = GridSearchCV(clfl2, param_grid=parameters, cv=5, scoring="accuracy")
fitmodel.fit(train.drop(['Survived', 'PassengerId', 'Name', 'Ticket', 'Fare', 'Cabin', 'Embarked'], axis = 1), train['Survived'])
fitmodel.best_estimator_, fitmodel.best_params_, fitmodel.best_score_, fitmodel.grid_scores_

(RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=5, max_features='auto', max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=1,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False),
 {'max_depth': 5, 'n_estimators': 400},
 0.83277216610549942,
 [mean: 0.74186, std: 0.04156, params: {'n_estimators': 10, 'max_depth': 1},
  mean: 0.76431, std: 0.01356, params: {'n_estimators': 100, 'max_depth': 1},
  mean: 0.76094, std: 0.02692, params: {'n_estimators': 200, 'max_depth': 1},
  mean: 0.75084, std: 0.02758, params: {'n_estimators': 300, 'max_depth': 1},
  mean: 0.75645, std: 0.02709, params: {'n_estimators': 400, 'max_depth': 1},
  mean: 0.75645, std: 0.02709, params: {'n_estimators': 500, 'max_depth': 1},
  mean: 0.80808, std: 0.01743, params: {'n_estimators': 10, 'max_depth': 2},
  mean: 0.80808, std: 0.01598, p

In [115]:
?RandomForestClassifier

In [86]:
string_array = np.array([train['CabinCategory'], train['Title']])
column_list = ['CabinCategory', 'Title']

le = preprocessing.LabelEncoder()
for column in range(len(train_string_array)):
    le.fit(string_array[column])
    train[column_list[column]] = le.transform(train[column_list[column]])
    train[column_list[column]].astype(int)

In [90]:
cv_score(LogisticRegression, train.drop(['Survived'], axis = 1), train['Survived'])

IndexError: indices are out-of-bounds

In [33]:
le = preprocessing.LabelEncoder()
for column in range(len(train_string_array)):
    le.fit(train_string_array[column])
    X_train[column_list[column]] = le.transform(X_train[column_list[column]])
    X_train[column_list[column]].astype(int)

le = preprocessing.LabelEncoder()
for column in range(len(CV_string_array)):
    le.fit(CV_string_array[column])
    X_CV[column_list[column]] = le.transform(X_CV[column_list[column]])
    X_CV[column_list[column]].astype(int)
#le.fit(X_train['CabinCategory'])
#X_train['CabinCategory'] = le.transform(X_train['CabinCategory'])

#le = preprocessing.LabelEncoder()
#le.fit(X_train['Title'])
#X_train['Title'] = le.transform(X_train['Title'])
#X_train[['CabinCategory', 'Title']] = X_train[['CabinCategory', 'Title']].astype(int)

In [34]:
print X_train.head()
print X_CV.head()

     PassengerId  Pclass                                               Name  \
240          241       3                              Zabour, Miss. Thamine   
53            54       2  Faunthorpe, Mrs. Lizzie (Elizabeth Anne Wilkin...   
578          579       3                   Caram, Mrs. Joseph (Maria Elias)   
847          848       3                                 Markoff, Mr. Marin   
842          843       1                            Serepeca, Miss. Augusta   

     Sex  Age  SibSp  Parch  Ticket     Fare Cabin Embarked  FamSize  \
240    1   30      1      0    2665  14.4542   NaN        C        2   
53     1   29      1      0    2926  26.0000   NaN        S        2   
578    1   30      1      0    2689  14.4583   NaN        C        2   
847    0   35      0      0  349213   7.8958   NaN        C        1   
842    1   30      0      0  113798  31.0000   NaN        C        1   

     CabinCategory CabinNumber  Title  Child  
240              2           0      7      0 

In [35]:
train_ids = X_train['PassengerId']
train_final = X_train.drop(['PassengerId', 'Name', 'Ticket', 'Fare', 'Cabin', 'Embarked'], axis = 1).astype(float).values

CV_ids = X_CV['PassengerId']
CV_final = X_CV.drop(['PassengerId', 'Name', 'Ticket', 'Fare', 'Cabin', 'Embarked'], axis = 1).astype(float).values

In [72]:
print train_final
print CV_final
print train_final.shape
print Y_train.shape

[[  3.   1.  30. ...,   0.   7.   0.]
 [  2.   1.  29. ...,   0.  11.   0.]
 [  3.   1.  30. ...,   0.  11.   0.]
 ..., 
 [  1.   0.  35. ...,  24.  10.   0.]
 [  3.   0.  32. ...,   0.  10.   0.]
 [  1.   0.  31. ...,  24.  10.   0.]]
[[  3.   0.  30. ...,   0.   6.   0.]
 [  3.   0.  30. ...,   0.   6.   0.]
 [  3.   0.  35. ...,   0.   6.   0.]
 ..., 
 [  3.   0.  40. ...,   0.   6.   0.]
 [  2.   1.  41. ...,   0.   7.   0.]
 [  3.   0.  30. ...,   0.   6.   0.]]
(596, 10)
(596,)


In [73]:
train_rf = RandomForestClassifier(n_estimators = 400, max_depth= 5)
train_rf = train_rf.fit(train_final, Y_train)

In [74]:
Y_CV_guess = pd.Series(train_rf.predict(CV_final).astype(int), index = Y_CV.index, name = 'Guess')
output = train_rf.predict(CV_final).astype(int)

In [75]:
table = pd.concat([Y_CV_guess, Y_CV], axis = 1)
print table['Guess'][table['Guess'] == table['Survived']].count().astype(float)/table['Guess'].count().astype(float)

0.840677966102


In [225]:
le = preprocessing.LabelEncoder()
le.fit(test_copy['CabinCategory'])
test_copy['CabinCategory'] = le.transform(test_copy['CabinCategory'])

le = preprocessing.LabelEncoder()
le.fit(test_copy['Title'])
test_copy['Title'] = le.transform(test_copy['Title'])
test_copy[['CabinCategory', 'Title']] = test_copy[['CabinCategory', 'Title']].astype(int)

In [226]:
test_ids = test_copy['PassengerId']
test_final = test_copy.drop(['PassengerId', 'Name', 'Ticket', 'Fare', 'Cabin', 'Embarked'], axis = 1).astype(float).values

In [227]:
output = train_rf.predict(test_final).astype(int)
output[0:5]

array([0, 0, 0, 0, 1])

In [228]:
predictions_file = open("myfirstforest.csv", "wb")
open_file_object = csv.writer(predictions_file)
open_file_object.writerow(["PassengerId","Survived"])
open_file_object.writerows(zip(test_ids, output))
predictions_file.close()